In [0]:
import requests
import json
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pyspark
from pyspark.sql.functions import sum,avg,max,min,mean,count

In [0]:
spark.conf.set(
  "")
spark.conf.set(
  "")

In [0]:
def gradingSentiment(score):
    if(score == None):
      return 0
    if(float(score)>0.5):
      return 1
    else:
      return 0

In [0]:
tablename = ""
month = "feb"
startDate = 1
endDate = 1
for i in range(startDate, endDate+1):
  intermeiatefilename = "wasbs://"+month+"@sentimentdatacs4225.blob.core.windows.net/"+month+"-sentiment-" + str(i) + ".csv"
  df = spark.read.csv(intermeiatefilename)
  split_col = pyspark.sql.functions.split(df['_c4'], ',')
  df2 = df.withColumn('postive', split_col.getItem(0)).withColumn('neutral', split_col.getItem(1)).withColumn('negative', split_col.getItem(2))
  udfsomefunc = F.udf(gradingSentiment, IntegerType())
#   df2.show()
  df2 = df2.withColumn('positiveCount', udfsomefunc(df2.postive))
  df2 = df2.withColumn('negativeCount', udfsomefunc(df2.negative))
  df2 = df2.withColumn('neturalCount', udfsomefunc(df2.neutral))
  df2 = df2.groupby("_c0","_c2").agg(sum("positiveCount").alias("positive_Count")
                                     ,sum("negativeCount").alias("negative_Count")
                                     ,sum("neturalCount").alias("netural_Count"))
  df2.write.format("jdbc") \
    .mode("append") \
    .option("url", "") \
    .option("dbtable", tablename) \
    .option("user", "") \
    .option("password", "") \
    .save()